In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

In [22]:
# pip install evaluate
# pip install sacrebleu
# pip install git+https://github.com/google-research/bleurt.git
import evaluate

In [44]:
# Update these paths if your files live in a different folder
files = {
    "sentence_npi_Deva": "/kaggle/input/floresdata/Data/flores/flores200_dataset/dev/npi_Deva.dev",
    "sentence_eng_Latn": "/kaggle/input/floresdata/Data/flores/flores200_dataset/dev/eng_Latn.dev",
    "sentence_sin_Sinh": "/kaggle/input/floresdata/Data/flores/flores200_dataset/dev/sin_Sinh.dev",
}

# Read each file into a list of lines
data = {}
for col, path in files.items():
    with open(path, "r", encoding="utf-8") as f:
        data[col] = [line.strip() for line in f]

# Truncate all lists to the same (shortest) length, to align triples
min_len = min(len(v) for v in data.values())
for col in data:
    data[col] = data[col][:min_len]

# Build a DataFrame
df = pd.DataFrame(data)

In [6]:
MODEL_ID = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID).eval()
device    = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

2025-04-24 20:44:04.131686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745527444.584825      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745527444.707597      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [11]:
# 4. Prepare the forced-BOS token for English
eng_bos = tokenizer.convert_tokens_to_ids("eng_Latn")
sin_bos = tokenizer.convert_tokens_to_ids("sin_Sinh")
nep_bos = tokenizer.convert_tokens_to_ids("npi_Deva")

In [7]:
def translate(
    text: str,
    bos_token_id: int,
    max_length: int = 128,
    num_beams: int = 4
) -> str:
    """
    Translate a single string using a forced BOS token.
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    outs   = model.generate(
        **inputs,
        forced_bos_token_id=bos_token_id,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True
    )
    return tokenizer.batch_decode(outs, skip_special_tokens=True)[0]

In [8]:
# # 4. Translation function
# def nep_to_eng(text):
#     inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
#     out = model.generate(
#         **inputs,
#         forced_bos_token_id=eng_bos,
#         max_length=128,
#         num_beams=4,
#         early_stopping=True
#     )
#     return tokenizer.batch_decode(out, skip_special_tokens=True)[0]

In [14]:
# def eng_to_sin(text: str) -> str:
#     inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
#     outs   = model.generate(
#         **inputs,
#         forced_bos_token_id=sin_bos,
#         max_length=128,
#         num_beams=4,
#         early_stopping=True
#     )
#     return tokenizer.batch_decode(outs, skip_special_tokens=True)[0]

In [8]:
def nep_to_eng(text: str) -> str:
    return translate(text, eng_bos)

def eng_to_sin(text: str) -> str:
    return translate(text, sin_bos)
    
def sin_to_eng(text: str) -> str:
    return translate(text, eng_bos)

def eng_to_nep(text: str) -> str:
    return translate(text, nep_bos)

In [45]:
# 6. Apply to your DataFrame (this may take a minute for large dfs)
df["nep_to_eng"] = df["sentence_npi_Deva"].map(nep_to_eng)

In [46]:
df["eng_to_sin"] = df["nep_to_eng"].map(eng_to_sin)

In [49]:
df['sin_to_eng']=df["sentence_sin_Sinh"].map(sin_to_eng)

In [50]:
df['eng_to_nep']=df["sin_to_eng"].map(eng_to_nep)

In [51]:
df.head()

,sentence_npi_Deva,sentence_eng_Latn,sentence_sin_Sinh,nep_to_eng,eng_to_sin,sin_to_eng,eng_to_nep
0,"सोमबारका दिन, स्ट्यानफोर्ड युनिभर्सिटी स्कुल अ...","On Monday, scientists from the Stanford Univer...","සඳුදා දින, ස්ටැන්ෆර්ඩ් සරසවි වෛද්‍ය පාසලේ විද්...","On Monday, scientists at the Stanford Universi...","සඳුදා, ස්ටැන්ෆර්ඩ් විශ්වවිද් යාලයේ වෛද් ය විද්...","On Monday, scientists at the Stanford Universi...","सोमबार, स्टानफोर्ड विश्वविद्यालयको मेडिकल स्कू..."
1,धनी देशहरूको तुलनामा कम आय भएका देशहरूमा क्यान...,Lead researchers say this may bring early dete...,"ප්‍රධාන පෙළේ පර්යේෂකයන් පවසන පරිදි මෙමගින්, පි...",Major researchers say that breast cancer survi...,"ප් රධාන පර්යේෂකයන් පවසන්නේ පිළිකා, කුරුලෑ, එච්...","According to leading researchers, this will en...","अग्रणी शोधकर्ताहरूका अनुसार, यसले कम आय भएका द..."
2,JAS 39C ग्रिपेन स्थानीय समयअनुसार बिहान 9:30 ब...,The JAS 39C Gripen crashed onto a runway at ar...,JAS 39C ග්‍රිපන් යානය දේශීය වේලාවෙන් පෙ.ව.9:30...,JAS 39C Gripen crashed at 9:30 am local time (...,ජේඑස් 39 සී ග් රිපෙන් ප් රාදේශීය වේලාවෙන් උදෑස...,The JAS 39C aircraft crashed into the runway a...,जेएएस ३९सी विमान स्थानीय समय अनुसार राति ९ः३० ...
3,पाइलटलाई स्क्वाड्रोन नेता डिलोक्रिट पट्टावीको ...,The pilot was identified as Squadron Leader Di...,බලඝන නායක ඩිලෝක්‍රිට් පටවී ගුවන් නියමුවා බව හඳ...,The pilot was identified as the leader of the ...,ගුවන් නියමුවෙකු ලෙස හඳුනාගෙන ඇත්තේ අංශයේ නායක ...,"The most powerful leader, Dilokrit Patvi, has ...","सबैभन्दा शक्तिशाली नेता, दिलोकृत पाट्वीलाई पाइ..."
4,स्थानीय मिडियाले प्रतिक्रियाको क्रममा विमानस्थ...,Local media reports an airport fire vehicle ro...,දේශීය මාධ්‍ය වාර්තා කළඒ සම්බන්ධව කටයුතු කරත් ද...,Local media responded by informing the airport...,ප් රාදේශීය මාධ් ය ප් රතිචාර දක්වමින් ගුවන් තොට...,Local media reported that the firefighting veh...,स्थानीय मिडियाले विमानस्थलमा रहेको अग्निशमन वा...


**Using devtest data**

In [ ]:
# load other metrics too
bleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")
bleurt = evaluate.load("bleurt", module_type="metric")

In [30]:
# 1. Gather references & hypotheses
refs = df["sentence_sin_Sinh"].tolist()
hyps = df["eng_to_sin"].tolist()

In [31]:
# compute each
res_bleu  = bleu.compute(predictions=hyps, references=[[r] for r in refs])
res_chrf  = chrf.compute(predictions=hyps, references=[[r] for r in refs])
res_bleurt = bleurt.compute(predictions=hyps, references=refs)

print(f"SacreBLEU = {res_bleu['score']:.2f}")
print(f"ChrF      = {res_chrf['score']:.2f}")
print(f"BLEURT    = {sum(res_bleurt['scores'])/len(res_bleurt['scores']):.4f}")


SacreBLEU = 8.87
ChrF      = 40.03
BLEURT    = 0.4092


In [42]:
refs = df["sentence_npi_Deva"].tolist()
hyps = df["eng_to_nep"].tolist()

In [43]:
# compute each
res_bleu  = bleu.compute(predictions=hyps, references=[[r] for r in refs])
res_chrf  = chrf.compute(predictions=hyps, references=[[r] for r in refs])
res_bleurt = bleurt.compute(predictions=hyps, references=refs)

print(f"SacreBLEU = {res_bleu['score']:.2f}")
print(f"ChrF      = {res_chrf['score']:.2f}")
print(f"BLEURT    = {sum(res_bleurt['scores'])/len(res_bleurt['scores']):.4f}")


SacreBLEU = 9.70
ChrF      = 43.20
BLEURT    = 0.0860


**Using dev data**

In [52]:
# 1. Gather references & hypotheses
refs = df["sentence_sin_Sinh"].tolist()
hyps = df["eng_to_sin"].tolist()

In [53]:
# compute each
res_bleu  = bleu.compute(predictions=hyps, references=[[r] for r in refs])
res_chrf  = chrf.compute(predictions=hyps, references=[[r] for r in refs])
res_bleurt = bleurt.compute(predictions=hyps, references=refs)

print(f"SacreBLEU = {res_bleu['score']:.2f}")
print(f"ChrF      = {res_chrf['score']:.2f}")
print(f"BLEURT    = {sum(res_bleurt['scores'])/len(res_bleurt['scores']):.4f}")


SacreBLEU = 8.62
ChrF      = 39.56
BLEURT    = 0.3894


In [54]:
refs = df["sentence_npi_Deva"].tolist()
hyps = df["eng_to_nep"].tolist()

In [55]:
# compute each
res_bleu  = bleu.compute(predictions=hyps, references=[[r] for r in refs])
res_chrf  = chrf.compute(predictions=hyps, references=[[r] for r in refs])
res_bleurt = bleurt.compute(predictions=hyps, references=refs)

print(f"SacreBLEU = {res_bleu['score']:.2f}")
print(f"ChrF      = {res_chrf['score']:.2f}")
print(f"BLEURT    = {sum(res_bleurt['scores'])/len(res_bleurt['scores']):.4f}")


SacreBLEU = 9.01
ChrF      = 43.94
BLEURT    = 0.0877
